In [ ]:
!pip install -U adapters
from adapters import AutoAdapterModel
from transformers import AutoTokenizer
import torch
model = AutoAdapterModel.from_pretrained("allenai/specter2_base")
adapter_name = model.load_adapter("allenai/specter2", source="hf", set_active=True)


In [ ]:

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')

#load base model
model = AutoAdapterModel.from_pretrained('allenai/specter2_base')

#load the adapter(s) as per the required task, provide an identifier for the adapter in load_as argument and activate it
model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)




In [ ]:
import pandas as pd
dataset=pd.read_json('submission_metadata.json').T
dataset.index.name = 'row_id'
dataset

In [ ]:
subset_dataset = dataset

papers = subset_dataset

# concatenate title and abstract
text_batch = [d['title'] + tokenizer.sep_token + (d.get('abstract') or '') for index,d in papers.iterrows()]

# preprocess the input
maliste=[]
i=0

model.eval()

# 2. Disable gradient calculation (saves Memory)
with torch.no_grad():
  for elt in text_batch:
    print(i)
    i+=1
    inputs = tokenizer(elt, padding=True, truncation=True,
                                    return_tensors="pt", return_token_type_ids=False, max_length=512)
    output = model(**inputs)
  # take the first token in the batch as the embedding
    maliste.append(output.last_hidden_state[:, 0, :].to('cpu'))
    del inputs,output

In [ ]:
import torch
print(len(maliste))
stacked = torch.stack(maliste, dim=0)
stacked.shape
stacked=stacked.squeeze(1)
stacked.shape

In [ ]:
torch.save(stacked,'papers.pt')